# Azure AI Agent Basic Example

This notebook demonstrates basic usage of AzureAIAgentClient to create agents with automatic lifecycle management. It shows both streaming and non-streaming responses with function tools.

## Features Covered:
- Creating an Azure AI Agent with automatic lifecycle management
- Using function tools (weather function)
- Non-streaming responses (get complete result at once)
- Streaming responses (get results as they are generated)
- Authentication using Azure CLI credentials

## Prerequisites

Before running this notebook, make sure you have:
1. Installed the agent-framework packages
2. Authenticated with Azure CLI (`az login --use-device-code`)
3. Configured your Azure AI services

## Import Required Libraries

First, let's import all the necessary libraries and modules:

In [ ]:
#!about
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.*"
#r "nuget: Microsoft.Agents.AI.AzureAI, 1.0.0-preview.*"
#r "nuget: dotenv.net"

using Azure.Identity;
using Microsoft.Agents.AI;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using Microsoft.Extensions.AI;
using dotenv.net;
using System.IO;
using System.Threading.Tasks;
using System.ComponentModel;
using System.Text.Json;

## Initial Setup
We'll start by loading environment variables and initializing an **AIProjectClient** so we can do all the agent-related actions. Let's do it! 🎉

In [ ]:
DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".", "..", "..","..", ".env") })); 
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeploymentName = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
Console.WriteLine($"TenantId: {tenantId}");
Console.WriteLine($"Project Endpoint: {projectEndpoint}");
// Create the Credentials
var credentialOptions = new InteractiveBrowserCredentialOptions
{
    TenantId = tenantId
};
var credential = new InteractiveBrowserCredential(credentialOptions);

AIProjectClient aiProjectClient = new AIProjectClient(new Uri(projectEndpoint),credential);

## Define Function Tools

Function tools allow the agent to call specific functions to gather information or perform actions. Here we define a simple weather function that the agent can use:

In [ ]:
[Description("Get the current weather for a given location.")]
string GetWeather([Description("The location to get the weather for.")]string location)
{
    // Simulate getting weather data
    var conditions = new[] { "sunny", "cloudy", "rainy", "windy", "stormy" };
    var random = new Random();
    var condition = conditions[random.Next(conditions.Length)];
    
    return $"🌤️ The weather in {location} is {condition} with a high of {random.Next(60, 101)}°F.";
}

## Non-Streaming Response Example

In this example, we'll create an agent and get a complete response at once (non-streaming). The agent will be automatically created and deleted after getting the response:

In [ ]:
async Task NonStreamingExample()
{
    // Example Non-streaming response (returns the full response at once)
    Console.WriteLine("--- Non-Streaming Response Example ---");

    var tool = AIFunctionFactory.Create(GetWeather);

    var agent = await aiProjectClient.CreateAIAgentAsync(
        name: "WeatherAgent",
        instructions: "You are a helpful weather agent.",
        model: modelDeploymentName!,
        tools: [tool]
    );

    var query = "What's the weather like in Seattle?";
    Console.WriteLine($"👤 User: {query}");
    var thread = agent.GetNewThread();
    var response = await agent.RunAsync(query, thread);
    Console.WriteLine($"🤖 Agent: {response}");
}

## Streaming Response Example

In this example, we'll demonstrate streaming responses where we get results as they are generated by the agent:

In [ ]:
async Task StreamingExample()
{
    // Example streaming response (returns response in chunks)
    Console.WriteLine("--- Streaming Response Example ---");

    var tool = AIFunctionFactory.Create(GetWeather);

    var agent = await aiProjectClient.CreateAIAgentAsync(
        name: "WeatherAgent",
        instructions: "You are a helpful weather agent.",
        model: modelDeploymentName!,
        tools: [tool]
    );

    var query = "What's the weather like in Seattle?";
    Console.WriteLine($"👤 User: {query}");
    var thread = agent.GetNewThread();
    Console.Write($"🤖 Agent: ");
    await foreach (AgentRunResponseUpdate update in agent.RunStreamingAsync(query, thread))
    {
        Console.Write(update);
    }
    Console.WriteLine(string.Empty);
}

## Main Execution Function

This function orchestrates the execution of both examples:

In [ ]:
async Task RunExamples()
{
    await NonStreamingExample();
    await StreamingExample();
}

## Run the Examples

Execute the main function to run both streaming and non-streaming examples:

In [ ]:
await RunExamples();

## Key Takeaways

1. **Automatic Lifecycle Management**: When no Agent ID is provided, the agent is automatically created and cleaned up
2. **Function Tools**: Agents can use custom functions to perform specific tasks (like getting weather data)
3. **Authentication**: Uses Azure CLI credentials for authentication (make sure to run `az login` first)
4. **Response Types**: 
   - Non-streaming: Get complete response at once using `agent.RunAsync()`
   - Streaming: Get response chunks as they're generated using `agent.RunStreamingAsync()`